# MorphCT Example Workflow

1. Start with an atomistic snapshot
2. Determine which atom indices belong to which chromophore using [SMARTS](https://www.daylight.com/dayhtml/doc/theory/theory.smarts.html) matching
3. Calculate the energies for each chromophore and chromophore pair using quantum chemical calculations (QCC)
4. Run the kinetic monte carlo (KMC) algorithm to calculate charge mobility

First let's import necessary modules:

In [1]:
import gsd.hoomd
import numpy as np

from morphct.chromophores import amber_dict, get_chromo_ids_smiles
from morphct.system import System

In the cell below, we'll create a system object for morphct. This the main class that will hold all the information for our simulation. We'll need to give it a gsd file, path to an output directory, the frame of the gsd file to use, the scaling factor to convert the lengths in the gsd to Angstroms, and a dictionary to map particle types to elements. Here's our starting structure, an atomistic (not coarse-grain or united atom) gsd file with 2 p3ht 16-mers: 

In [2]:
gsdfile = "/home/stephaniemccallu/scratch/PPS/Entanglements/repos/morphct/last_frame_eq.gsd"
#"/home/stephaniemccallu/scratch/PPS/Entanglements/repos/back-mapped-pps.gsd"

system = System(gsdfile, "output_pps_rerun_2", frame=-1, scale=3.5636, conversion_dict=amber_dict)
system.visualize_system()

/home/stephaniemccallu/miniforge3/envs/morphct-new/lib/python3.7/site-packages/mbuild/compound.py:553: UserWarning: Some particle of <C pos=([11.9603 12.9552 11.7457]), 3 bonds, id: 139843473978640> does not have a charge.They will not be accounted for during this calculation.
  f"Some particle of {self} does not have a charge."
/home/stephaniemccallu/miniforge3/envs/morphct-new/lib/python3.7/site-packages/mbuild/compound.py:553: UserWarning: Some particle of <C pos=([12.0369 12.9219 11.8574]), 3 bonds, id: 139843423786256> does not have a charge.They will not be accounted for during this calculation.
  f"Some particle of {self} does not have a charge."
/home/stephaniemccallu/miniforge3/envs/morphct-new/lib/python3.7/site-packages/mbuild/compound.py:553: UserWarning: Some particle of <C pos=([12.039  13.0068 11.9651]), 3 bonds, id: 139843423786960> does not have a charge.They will not be accounted for during this calculation.
  f"Some particle of {self} does not have a charge."
/home/s

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Next let's use SMARTS matching to detect our chromophores. This SMARTS string shown below is for p3ht. 

Note: The positions/orientations in the final frame of the gsd file are not optimal, so we are using the first frame (before any distortion) for SMARTS matching and then mapping those indices to the final structure. Sometimes SMARTS matching is not the best method to find the chromophores--check the ITIC example to see another method.

In [3]:
smarts_str = "c1ccc(S)cc1"

with gsd.hoomd.open(gsdfile) as f:
    snap0 = f[0]

aaids = get_chromo_ids_smiles(snap0, smarts_str, system.conversion_dict)

Found 190 chromophores.


Next let's add these chromophores to the system and visualize them. We only have donor species in this system, and they'll be colored purple.

In [4]:
# the indices of the donor in one molecule
# saved for use in morphct-flow
np.savetxt("pps_d_ids_rerun.csv", aaids[:16], fmt="%i")

In [5]:
system.add_chromophores(aaids, "donor",chromophore_kwargs={'reorganization_energy':0.29757})

system.visualize_chromophores()

/home/stephaniemccallu/miniforge3/envs/morphct-new/lib/python3.7/site-packages/mbuild/compound.py:553: UserWarning: Some particle of <I pos=([11.9603 12.9552 11.7457]), 3 bonds, id: 139843401545168> does not have a charge.They will not be accounted for during this calculation.
  f"Some particle of {self} does not have a charge."
/home/stephaniemccallu/miniforge3/envs/morphct-new/lib/python3.7/site-packages/mbuild/compound.py:553: UserWarning: Some particle of <I pos=([12.0369 12.9219 11.8574]), 3 bonds, id: 139843401545296> does not have a charge.They will not be accounted for during this calculation.
  f"Some particle of {self} does not have a charge."
/home/stephaniemccallu/miniforge3/envs/morphct-new/lib/python3.7/site-packages/mbuild/compound.py:553: UserWarning: Some particle of <I pos=([12.039  13.0068 11.9651]), 3 bonds, id: 139843401544848> does not have a charge.They will not be accounted for during this calculation.
  f"Some particle of {self} does not have a charge."
/home/s

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Next let's compute the energies required to run the KMC simulation. First, the neighbors will be calculated (using voronoi analysis) and then the single and dimer energies will be calculated and saved to a file. (So that the simulation can be restarted from this point more easily.)

In [6]:
system.compute_energies()

There are 1608 chromophore pairs
Starting singles energy calculation...
Finished in 50.82 s. Output written to output_pps_rerun_2/singles_energies.txt.
Starting dimer energy calculation...
Finished in 345.44 s. Output written to output_pps_rerun_2/dimer_energies.txt.


We can check that the pair and singles inputs look reasonable. There won't be any bonds and hydrogen atoms should've been added.

In [7]:
i = 86 # try any number from 0 to 191
print(f"Pair #{i}:")
system.visualize_qcc_input(i, single=False)

i = 0 # try any number from 0 to 31
print(f"Single #{i}:")
system.visualize_qcc_input(i)

Pair #86:


/home/stephaniemccallu/miniforge3/envs/morphct-new/lib/python3.7/site-packages/mbuild/compound.py:553: UserWarning: Some particle of <C pos=([0.2902 0.1008 0.2502]), 0 bonds, id: 139843183792528> does not have a charge.They will not be accounted for during this calculation.
  f"Some particle of {self} does not have a charge."
/home/stephaniemccallu/miniforge3/envs/morphct-new/lib/python3.7/site-packages/mbuild/compound.py:553: UserWarning: Some particle of <C pos=([0.201  0.1978 0.2259]), 0 bonds, id: 139843180393744> does not have a charge.They will not be accounted for during this calculation.
  f"Some particle of {self} does not have a charge."
/home/stephaniemccallu/miniforge3/envs/morphct-new/lib/python3.7/site-packages/mbuild/compound.py:553: UserWarning: Some particle of <C pos=([0.0608 0.1623 0.2281]), 0 bonds, id: 139843180392720> does not have a charge.They will not be accounted for during this calculation.
  f"Some particle of {self} does not have a charge."
/home/stephaniem

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Single #0:


/home/stephaniemccallu/miniforge3/envs/morphct-new/lib/python3.7/site-packages/mbuild/compound.py:553: UserWarning: Some particle of <C pos=([ 0.0131 -0.0907 -0.1439]), 0 bonds, id: 139843154873104> does not have a charge.They will not be accounted for during this calculation.
  f"Some particle of {self} does not have a charge."
/home/stephaniemccallu/miniforge3/envs/morphct-new/lib/python3.7/site-packages/mbuild/compound.py:553: UserWarning: Some particle of <C pos=([ 0.0897 -0.124  -0.0322]), 0 bonds, id: 139843154873296> does not have a charge.They will not be accounted for during this calculation.
  f"Some particle of {self} does not have a charge."
/home/stephaniemccallu/miniforge3/envs/morphct-new/lib/python3.7/site-packages/mbuild/compound.py:553: UserWarning: Some particle of <C pos=([ 0.0918 -0.0391  0.0755]), 0 bonds, id: 139843154873424> does not have a charge.They will not be accounted for during this calculation.
  f"Some particle of {self} does not have a charge."
/home/s

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Once the energy files are finished, we can use them to set the energy values of the chromophores . (This can also be run to restart the calculation from this point.)

In [8]:
system.set_energies()

Energies set.


This function sets the `homo_1`, `homo`, `lumo`, `lumo_1`, `neighbors_delta_e`, and `neighbors_ti` attributes of each chromphore:

In [9]:
i = 0
chromo = system.chromophores[i]
print(f"Chromophore {i}:")
print(
    f"HOMO-1: {chromo.homo_1:.2f} HOMO: {chromo.homo:.2f} LUMO: {chromo.lumo:.2f} "
    f"LUMO+1: {chromo.lumo_1:.2f}"
)
print(f"{len(chromo.neighbors)} neighbors")
print(f"DeltaE of first neighbor: {chromo.neighbors_delta_e[0]:.3f}")
print(f"Transfer integral of first neighbor: {chromo.neighbors_ti[0]:.3f}")

Chromophore 0:
HOMO-1: -9.28 HOMO: -8.49 LUMO: 0.80 LUMO+1: 0.97
21 neighbors
DeltaE of first neighbor: -0.239
Transfer integral of first neighbor: 0.096


With all the energy values set, we're ready to run KMC! We need to set the temperature that the KMC simulation will be run at and the lifetimes and numbers of our carriers:

In [10]:
lifetimes = [1e-14, 1e-13, 1e-12, 1e-11]
temp = 300
system.run_kmc(lifetimes, temp, n_holes=10, verbose=1)

All KMC jobs completed!
Combining outputs...
---------- KMC_ANALYZE ----------
All figures saved in output_pps_rerun_2/kmc/figures
---------------------------------
Considering the transport of hole...
Obtaining mean squared displacements...
	Notice: The data from 19 carriers were
	discarded due to the carrier lifetime being more than double
	(or less than half of) the specified carrier lifetime.
Plotting distribution of hole displacements
	Figure saved as hole_displacement_dist.png
Calculating mobility...
	Standard Error 5.4173553981017445e-08
	Fitting r_val = 0.9893110792353181
	Figure saved as lin_MSD_hole.png
	Figure saved as semi_log_MSD_hole.png
	Figure saved as log_MSD_hole.png
	----------------------------------------
	Hole mobility = 3.35E-02  +/- 5.92E-03 cm^2 V^-1 s^-1
	----------------------------------------
Calculating hole trajectory anisotropy...
	----------------------------------------
	Hole charge transport anisotropy: 0.125
	----------------------------------------


/home/stephaniemccallu/miniforge3/envs/morphct-new/lib/python3.7/site-packages/scipy/optimize/minpack.py:834: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


	Figure saved as donor_delta_E_ij.png
donor separation cluster cut-off set to 7.026906916997309
Neighbor histogram figure saved as neighbor_hist_donor.png
	Notice: No minima found in distribution. Cutoff set to None.
Orientation histogram figure saved as orientation_hist_donor.png
	Notice: No minima found in distribution. Cutoff set to None.
	Figure saved as donor_transfer_integral_mols.png
Examining the donor material...
Calculating clusters...
	No cutoff provided: cluster cutoff set to 6.485
	----------------------------------------
	Donor: Detected 1 total
	and 1 large clusters (size > 6).
	Largest cluster size: 190 chromophores.
	Ratio in "large" clusters: 1.00
	----------------------------------------
Examining the acceptor material...
	No material found. Continuing...
Mean intra-cluster donor rate: 8.070e+14+/-4.828e+12
	Figure saved as donor_hopping_rate_clusters.png
	Figure saved as donor_transfer_integral_clusters.png
Mean intra-molecular donor rate: 2.862e+15+/-3.219e+13
Mean

The output files for each process are saved in `output_p3ht/kmc/kmc_PROC#.log` (where `PROC#` is whatever process number the job was run on) and analysis plots will be saved in `output_p3ht/kmc/figures/`.

In [11]:
with open("output_pps_rerun_2/kmc/kmc_00.log", "r") as f:
    lines = f.readlines()
print(*lines)

Found 2 jobs to run
 starting job 0
 	hole hopped 16 times over 8.57e-14 seconds into image [ 0 -1 -1] for a displacement of
 	10.14 (took walltime 0.08 seconds)
 starting job 1
 	hole hopped 14 times over 7.98e-14 seconds into image [0 0 0] for a displacement of
 	4.83 (took walltime 0.08 seconds)

